In [10]:
import os
import cv2
import json
import numpy as np
from PIL import Image

MAX_N = 100

categories = [
  {
    "supercategory": "none",
    "name": "people",
    "id": 0
  }
]

def main():
  label_path = './Root/images/labels/0009'
  save_path = './output/'
  res_file = {
    "categories": categories,
    "images": [],
    "annotations": []
  }

  for label_filename in os.listdir(label_path):
    label = cv2.imread(f'{label_path}/{label_filename}')
    blue_channel = label[:, :, 0]
    output_filename = label_filename[:-4]
    image_id = 0
    img_elem = {"file_name": output_filename,
                "height": label.shape[0],
                "width": label.shape[1],
                "id": image_id}

    res_file["images"].append(img_elem)
    save_to_json(res_file, output_filename, blue_channel, label, image_id)
  with open(f'{save_path}/train.json', 'w') as out:
      out.write(json.dumps(res_file))
      print('Saved to json')

def save_to_json(res_file, filename, blue_channel, label, image_id):
      instance_IDs = np.unique(blue_channel)
      annot_count = 0
      for tracking_ID in instance_IDs:
        if tracking_ID != 0:
          c = blue_channel.copy()
          c = np.where(c == tracking_ID, 200, 0).astype(np.uint8)
          x, y, w, h = cv2.boundingRect(c)
          area = w * h
          poly = [[x, y],
                  [x + w, y],
                  [x + w, y + h],
                  [x, y + h]]
          annot_elem = {
            "id": annot_count,
            "bbox": [
                float(x),
                float(y),
                float(w),
                float(h)
            ],
            "segmentation": list([poly]),
            "image_id": image_id,
            "ignore": 0,
            "category_id": 0,
            "iscrowd": 0,
            "area": float(area)
          }
          res_file["annotations"].append(annot_elem)

if __name__ == '__main__':
    main()



Saved to json
